In [1]:
import pickle
import numpy as np
from keras import backend as K
from matplotlib import pyplot
from sklearn.model_selection import KFold
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
import tensorflow as tf


participantCount = 32

rawData = [{}]*participantCount

for i in range(participantCount):
    rawData[i] = pickle.load(open(f'../data/s{i+1:02}.dat', 'rb'), encoding="latin1")

labels = np.array(list(map(lambda participant : participant['labels'], rawData)))
data = np.array(list(map(lambda participant : participant['data'], rawData)))

def get_y(emotion):
    return (labels.reshape(-1, 4)[:,emotion] >= 5).astype(int)

def get_x():
    return data.reshape(-1, 40, 8064)

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
X = get_x()[:1]
X = np.moveaxis(X, 1, 2)[0]
np.savetxt("foo.csv", X, delimiter=",")
np.loadtxt("foo.csv", delimiter=",")

In [ ]:
physical_devices = tf.config.list_physical_devices('CPU')
# tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

Y = get_y(0)
X = np.moveaxis(get_x(), 1, 2)

for i in [0, 1, 3]:
    print("EMOTION NUMBER", i)
    Y = get_y(i)
    kfold = KFold(n_splits=5, shuffle=False)
    cvscores = []
    fscores = []
    for train, test in kfold.split(X, Y):
        model = Sequential()
        model.add(LSTM(80, input_dim=40))
        model.add(Dense(1, activation='sigmoid'))
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=["acc", f1_m, precision_m, recall_m])

        model.fit(X[train], Y[train], validation_data=(X[test], Y[test]), epochs=100, batch_size=128, verbose=2, shuffle=False)

        scores = model.evaluate(X[test], Y[test], verbose=0)
        print("%s: %.2f%%, %s: %.2f%%" % (model.metrics_names[1], scores[1]*100, model.metrics_names[2], scores[2]*100))
        cvscores.append(scores[1] * 100)
        fscores.append(scores[2] * 100)

    print("%.2f%% (+/- %.2f%%), %.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores), np.mean(fscores), np.std(fscores)))    
    print("EMOTION", i)

In [2]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

X = np.moveaxis(get_x(), 1, 2)

for i in [0, 1, 3]:
    print("EMOTION NUMBER", i)
    Y = get_y(i)
    model = Sequential()
    model.add(LSTM(80, input_dim=40))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=["acc", f1_m, precision_m, recall_m])
    model.fit(X, Y, epochs=100, batch_size=128, verbose=2, shuffle=False)
    model.save_weights('weights_' + str(i) + '.h5')

EMOTION NUMBER 0
Epoch 1/100
10/10 - 8s - loss: 0.7323 - acc: 0.5031 - f1_m: 0.6045 - precision_m: 0.5546 - recall_m: 0.7008
Epoch 2/100
10/10 - 4s - loss: 0.6992 - acc: 0.5367 - f1_m: 0.6326 - precision_m: 0.5810 - recall_m: 0.7301
Epoch 3/100
10/10 - 4s - loss: 0.6752 - acc: 0.5688 - f1_m: 0.6599 - precision_m: 0.6055 - recall_m: 0.7544
Epoch 4/100
10/10 - 4s - loss: 0.6616 - acc: 0.5891 - f1_m: 0.6806 - precision_m: 0.6156 - recall_m: 0.7874
Epoch 5/100
10/10 - 4s - loss: 0.6528 - acc: 0.6078 - f1_m: 0.6902 - precision_m: 0.6342 - recall_m: 0.7797
Epoch 6/100
10/10 - 4s - loss: 0.6437 - acc: 0.6250 - f1_m: 0.7029 - precision_m: 0.6464 - recall_m: 0.7882
Epoch 7/100
10/10 - 4s - loss: 0.6342 - acc: 0.6453 - f1_m: 0.7199 - precision_m: 0.6607 - recall_m: 0.8063
Epoch 8/100
10/10 - 4s - loss: 0.6301 - acc: 0.6586 - f1_m: 0.7275 - precision_m: 0.6707 - recall_m: 0.8080
Epoch 9/100
10/10 - 4s - loss: 0.6203 - acc: 0.6703 - f1_m: 0.7385 - precision_m: 0.6780 - recall_m: 0.8251
Epoch 10/10

10/10 - 4s - loss: 0.4843 - acc: 0.7555 - f1_m: 0.7924 - precision_m: 0.7562 - recall_m: 0.8367
Epoch 77/100
10/10 - 4s - loss: 0.4869 - acc: 0.7523 - f1_m: 0.7908 - precision_m: 0.7527 - recall_m: 0.8382
Epoch 78/100
10/10 - 4s - loss: 0.4896 - acc: 0.7500 - f1_m: 0.7883 - precision_m: 0.7526 - recall_m: 0.8332
Epoch 79/100
10/10 - 4s - loss: 0.4862 - acc: 0.7523 - f1_m: 0.7881 - precision_m: 0.7578 - recall_m: 0.8265
Epoch 80/100
10/10 - 4s - loss: 0.4872 - acc: 0.7547 - f1_m: 0.7897 - precision_m: 0.7607 - recall_m: 0.8266
Epoch 81/100
10/10 - 4s - loss: 0.4893 - acc: 0.7547 - f1_m: 0.7878 - precision_m: 0.7646 - recall_m: 0.8187
Epoch 82/100
10/10 - 4s - loss: 0.4924 - acc: 0.7445 - f1_m: 0.7766 - precision_m: 0.7589 - recall_m: 0.8027
Epoch 83/100
10/10 - 4s - loss: 0.4903 - acc: 0.7508 - f1_m: 0.7889 - precision_m: 0.7548 - recall_m: 0.8339
Epoch 84/100
10/10 - 4s - loss: 0.4892 - acc: 0.7430 - f1_m: 0.7864 - precision_m: 0.7425 - recall_m: 0.8443
Epoch 85/100
10/10 - 4s - loss: 

Epoch 52/100
10/10 - 4s - loss: 0.5303 - acc: 0.7250 - f1_m: 0.7829 - precision_m: 0.7347 - recall_m: 0.8450
Epoch 53/100
10/10 - 4s - loss: 0.5283 - acc: 0.7234 - f1_m: 0.7844 - precision_m: 0.7351 - recall_m: 0.8501
Epoch 54/100
10/10 - 4s - loss: 0.5262 - acc: 0.7328 - f1_m: 0.7942 - precision_m: 0.7374 - recall_m: 0.8704
Epoch 55/100
10/10 - 4s - loss: 0.5248 - acc: 0.7281 - f1_m: 0.7890 - precision_m: 0.7349 - recall_m: 0.8600
Epoch 56/100
10/10 - 4s - loss: 0.5243 - acc: 0.7344 - f1_m: 0.7967 - precision_m: 0.7347 - recall_m: 0.8786
Epoch 57/100
10/10 - 4s - loss: 0.5198 - acc: 0.7344 - f1_m: 0.7953 - precision_m: 0.7375 - recall_m: 0.8725
Epoch 58/100
10/10 - 4s - loss: 0.5269 - acc: 0.7148 - f1_m: 0.7737 - precision_m: 0.7277 - recall_m: 0.8325
Epoch 59/100
10/10 - 4s - loss: 0.5234 - acc: 0.7188 - f1_m: 0.7785 - precision_m: 0.7322 - recall_m: 0.8375
Epoch 60/100
10/10 - 4s - loss: 0.5240 - acc: 0.7188 - f1_m: 0.7783 - precision_m: 0.7332 - recall_m: 0.8375
Epoch 61/100
10/10 

10/10 - 4s - loss: 0.5287 - acc: 0.7508 - f1_m: 0.8419 - precision_m: 0.7454 - recall_m: 0.9782
Epoch 28/100
10/10 - 4s - loss: 0.5253 - acc: 0.7563 - f1_m: 0.8455 - precision_m: 0.7499 - recall_m: 0.9794
Epoch 29/100
10/10 - 4s - loss: 0.5225 - acc: 0.7539 - f1_m: 0.8441 - precision_m: 0.7487 - recall_m: 0.9782
Epoch 30/100
10/10 - 4s - loss: 0.5184 - acc: 0.7586 - f1_m: 0.8473 - precision_m: 0.7517 - recall_m: 0.9815
Epoch 31/100
10/10 - 4s - loss: 0.5185 - acc: 0.7594 - f1_m: 0.8484 - precision_m: 0.7528 - recall_m: 0.9838
Epoch 32/100
10/10 - 4s - loss: 0.5163 - acc: 0.7609 - f1_m: 0.8492 - precision_m: 0.7546 - recall_m: 0.9825
Epoch 33/100
10/10 - 4s - loss: 0.5160 - acc: 0.7609 - f1_m: 0.8482 - precision_m: 0.7549 - recall_m: 0.9790
Epoch 34/100
10/10 - 4s - loss: 0.5138 - acc: 0.7625 - f1_m: 0.8485 - precision_m: 0.7566 - recall_m: 0.9767
Epoch 35/100
10/10 - 4s - loss: 0.5114 - acc: 0.7656 - f1_m: 0.8503 - precision_m: 0.7611 - recall_m: 0.9743
Epoch 36/100
10/10 - 4s - loss: 